In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr'

%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/mink/notebooks/CameraTraps')  # append this repo to PYTHONPATH
sys.path.append('/home/mink/lib/ai4eutils')

In [3]:
import json
import os
from collections import Counter, defaultdict
from random import sample
from shutil import copyfile
from multiprocessing.pool import ThreadPool
from datetime import datetime

import numpy as np
import pandas as pd
from tqdm import tqdm

import path_utils  # ai4eutils

from data_management.megadb.schema import sequences_schema_check
from data_management.megadb.converters.cct_to_megadb import process_sequences

# fws_butler_redrock

In [4]:
dataset_name = 'fws_butler_redrock'

container_root = '/mink_disk_0/camtraps/fws-butler/'  

path_to_output = f'/home/mink/camtraps/data/megadb_jsons/{dataset_name}.json' 
path_to_output_temp = f'/home/mink/camtraps/data/megadb_jsons/{dataset_name}_temp.json' 

## Step 0 - Add an entry to the `datasets` table

Done

## Step 1 - Prepare the `sequence` objects to insert into the database

Images are put into species folders at the bottom-most directory. Empty is denoted "Ghost". No sequence info recovered.

Locations are consistent across the two Redrock folders.

In [5]:
paths = path_utils.recursive_file_list(container_root)
len(paths)
paths = sorted([p.split(container_root)[1] for p in paths if path_utils.is_image_file(p)])
len(paths)
# some corrupted images (a few bytes in size) start with a . in the actual name - to filter out next

21634

21634

In [6]:
sequences = []
locations = set()

for p in tqdm(paths):
    basename = os.path.basename(p)
    if basename.startswith('.'):
        continue
    
    p_parts = p.split('/')
    
    assert len(p_parts) == 5 or len(p_parts) == 4, p
    
    location = p_parts[1]
    locations.add(location)
    
    if len(p_parts) == 5:
        species = p_parts[2].lower()
        if species == 'ghost':
            species = 'empty'
    else:
        species = '__label_unavailable'

    b_parts = basename.split('.')[0].split(' ')
    if len(b_parts) != 6:
        continue # 4 images says "Copy", 1 Unknown and 3 Bighorn
        
    timestamp = datetime(
        year=int(b_parts[0]),
        month=int(b_parts[1]),
        day=int(b_parts[2]),
        hour=int(b_parts[3]),
        minute=int(b_parts[4]),
        second=int(b_parts[5])
    )
    
    sequences.append({
        'dataset': dataset_name,
        'seq_id': f'dummy_redrock_{location}_{len(sequences)}',
        'location': location,
        'class': [species],
        'images': [{
            'file': p,
            'frame_num': 1,
            'datetime': str(timestamp)
        }]
    })
        
len(sequences)

100%|██████████| 21634/21634 [00:00<00:00, 76917.24it/s]


21587

In [7]:
datetime(year=2021, month=2, day=26, hour=1, minute=10, second=12)
str(datetime(year=2021, month=2, day=26, hour=1, minute=10, second=12))

datetime.datetime(2021, 2, 26, 1, 10, 12)

'2021-02-26 01:10:12'

In [8]:
len(locations)

24

In [9]:
sequences[-100]

{'dataset': 'fws_butler_redrock',
 'seq_id': 'dummy_redrock_Solar_21487',
 'location': 'Solar',
 'class': ['javelina'],
 'images': [{'file': 'RedrockPics2/Solar/Javelina/10/2017 12 24 14 47 47.JPG',
   'frame_num': 1,
   'datetime': '2017-12-24 14:47:47'}]}

## Step 2 - Pass the schema check

Once your metadata are in the MegaDB format for `sequence` items, we check that they conform to the format's schema.

If the format conforms, the following messages will be printed:

```
Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
```

For large datasets, the second step will take some time (~ a minute). 

Otherwise there will be an error message describing what's wrong. Please fix the issues until all checks are passed. You might need to write some snippets of code to loop through the `sequence` items to understand which entries have problems.

In [10]:
%%time

sequences_schema_check.sequences_schema_check(sequences)

Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
CPU times: user 2.88 s, sys: 428 µs, total: 2.88 s
Wall time: 2.88 s


In [11]:
with open(path_to_output_temp, 'w', encoding='utf-8') as f:
    json.dump(sequences, f, indent=1, ensure_ascii=False)

### Step 2b - copy images to flat folder

In [12]:
def copy_file(src_path, dst_path):
    return copyfile(src_path, dst_path)

In [13]:
%%time

path_pairs = []
for seq in tqdm(sequences):
    
    if 'empty' in seq['class']:
        continue
    
    seq_id = seq['seq_id']
    for im in seq['images']:
        src_path = os.path.join(container_root, im['file'])
        assert os.path.exists(src_path), src_path
        frame = 1
        dst_path = os.path.join('/mink_disk_0/camtraps/imerit12c', 
                                f'{dataset_name}.seq{seq_id}.frame{frame}.jpg')
        path_pairs.append((src_path, dst_path))

100%|██████████| 21587/21587 [00:00<00:00, 159790.55it/s]

CPU times: user 111 ms, sys: 28.1 ms, total: 139 ms
Wall time: 137 ms


In [14]:
len(path_pairs)
path_pairs[1000]

15680

('/mink_disk_0/camtraps/fws-butler/RedrockPics/G9/Bighorn/01/2017 09 01 14 15 20.JPG',
 '/mink_disk_0/camtraps/imerit12c/fws_butler_redrock.seqdummy_redrock_G9_2491.frame1.jpg')

In [15]:
%%time

with ThreadPool(8) as pool:
    dst_paths = pool.starmap(copy_file, path_pairs)

CPU times: user 13.6 s, sys: 45.9 s, total: 59.5 s
Wall time: 2min 57s


In [16]:
len(dst_paths)

15680